<a href="https://colab.research.google.com/github/melitadn/FGA-Python-Basic/blob/master/TGS_RecSys_Content_Based_movies_py_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://imgur.com/s4wTnl7.jpg" align = "center">

<h1 align=center><font size = 5>CONTENT-BASED FILTERING</font></h1>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.

### Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
- <a href="#ref1">Acquiring the Data</a><br />
- <a href="#ref2">Preprocessing</a><br />
- <a href="#ref3">Content-Based Filtering</a>
</div>
<br>

<a id="ref1"></a>
# Acquiring the Data

To acquire and extract the data, simply run the following Bash scripts:  
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/). Lets download the dataset. To download the data, we will use **`!wget`**. To download the data, we will use `!wget` to download it from IBM Object Storage.  
__Did you know?__ When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)

Now you're ready to start working with the data!

<a id="ref2"></a>
# Preprocessing

First, let's get all of the imports out of the way:

In [0]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:

In [112]:
#Storing the movie information into a pandas dataframe
anime_df = pd.read_csv('anime.csv')
# anime_df=
#Storing the user information into a pandas dataframe
rating_df = pd.read_csv('rating.csv')
# rating_df=
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [113]:
#Every genre is separated by a , so we simply have to call the split function on |
anime_df['genre'] = anime_df.genre.str.split(',')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,9.26,793665
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.25,114262
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,9.17,673572
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.16,151266


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.

In [118]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
animesWithGenres_df = anime_df.copy()
filter_df=animesWithGenres_df[~animesWithGenres_df['genre'].isnull()]
filter_df.head(2)



,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,9.26,793665


In [119]:
filter_anime=anime_df[~anime_df['genre'].isnull()]
filter_anime.head(2)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,9.26,793665


In [120]:
# # #For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in filter_anime.iterrows():
     for genres in row['genre']:
         filter_df.at[index, genres] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
filter_df = filter_df.fillna(0)
filter_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,anime_id,name,genre,type,episodes,rating,members,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,9.37,200630,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",TV,64,9.26,793665,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.25,114262,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",TV,24,9.17,673572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",TV,51,9.16,151266,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, let's look at the ratings dataframe.

In [121]:
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [126]:
#filter_anime.head(1)
# alternatively, delete columns using the columns parameter of drop
filter_anime = filter_anime.drop(columns="rating")
filter_anime.head(1)

,anime_id,name,genre,type,episodes,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,200630


In [127]:
filter_df = filter_df.drop(columns="rating")
filter_df.head(1)


,anime_id,name,genre,type,episodes,members,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,200630,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

<a id="ref3"></a>
# Content-Based recommendation system

Now, let's take a look at how to implement __Content-Based__ or __Item-Item recommendation systems__. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the __userInput__. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [134]:
userInput = [
            {'name':'Kimi no Na wa.', 'rating':7.4},
             {'name':'Under World', 'rating':3.5},
             {'name':'Taimanin Asagi 3', 'rating':8},
             {'name':'Ookami Kodomo no Ame to Yuki', 'rating':4.5}
         ] 
inputAnimes = pd.DataFrame(userInput)
inputAnimes

,name,rating
0,Kimi no Na wa.,7.4
1,Under World,3.5
2,Taimanin Asagi 3,8.0
3,Ookami Kodomo no Ame to Yuki,4.5


#### Add movieId to input user
With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [0]:
#filter_anime

In [138]:
#Filtering out the anime by name
inputId = filter_anime[filter_anime['name'].isin(inputAnimes['name'].tolist())]
inputId.head(4)

#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
#inputAnimes

,anime_id,name,genre,type,episodes,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,200630
18,12355,Ookami Kodomo no Ame to Yuki,"[Fantasy, Slice of Life]",Movie,1,226193
12285,34399,Taimanin Asagi 3,"[Demons, Hentai, Supernatural]",OVA,Unknown,485
12290,5543,Under World,[Hentai],OVA,1,183


In [139]:
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputAnimes = pd.merge(inputId, inputAnimes,how='left')
#Dropping information we won't use from the input dataframe
inputAnimes = inputAnimes.drop('genre', 1).drop('type', 1).drop('members',1).drop('episodes',1)

inputAnimes

,anime_id,name,rating
0,32281,Kimi no Na wa.,7.4
1,12355,Ookami Kodomo no Ame to Yuki,4.5
2,34399,Taimanin Asagi 3,8.0
3,5543,Under World,3.5


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.

In [141]:
#Filtering out the movies from the input
userAnimes = filter_df[filter_df['anime_id'].isin(inputAnimes['anime_id'].tolist())]
userAnimes

,anime_id,name,genre,type,episodes,members,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,200630,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,12355,Ookami Kodomo no Ame to Yuki,"[Fantasy, Slice of Life]",Movie,1,226193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12285,34399,Taimanin Asagi 3,"[Demons, Hentai, Supernatural]",OVA,Unknown,485,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12290,5543,Under World,[Hentai],OVA,1,183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.

In [142]:
#Resetting the index to avoid future issues
userAnimes = userAnimes.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userAnimes.drop('anime_id', 1).drop('name', 1).drop('genre', 1).drop('type', 1).drop('episodes', 1).drop('members',1)
userGenreTable

,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,Demons,Ecchi,Josei,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.

In [143]:
inputAnimes['rating']

0    7.4
1    4.5
2    8.0
3    3.5
Name: rating, dtype: float64

In [144]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputAnimes['rating'])
#The user profile
userProfile

Drama              7.4
 Romance           7.4
 School            7.4
 Supernatural     15.4
Action             0.0
 Adventure         0.0
 Drama             0.0
 Fantasy           0.0
 Magic             0.0
 Military          0.0
 Shounen           0.0
 Comedy            0.0
 Historical        0.0
 Parody            0.0
 Samurai           0.0
 Sci-Fi            0.0
Sci-Fi             0.0
 Thriller          0.0
Comedy             0.0
 Sports            0.0
 Super Power       0.0
 Space             0.0
 Slice of Life     4.5
 Mecha             0.0
Adventure          0.0
 Music             0.0
 Mystery           0.0
 Seinen            0.0
Fantasy            4.5
 Martial Arts      0.0
                  ... 
 Dementia          0.0
Cars               0.0
Mecha              0.0
Horror             0.0
School             0.0
 Kids              0.0
Historical         0.0
Kids               0.0
 Shoujo Ai         0.0
Shounen            0.0
Shoujo             0.0
Magic              0.0
Harem      

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.

Let's start by extracting the genre table from the original dataframe:

In [145]:
#Now let's get the genres of every movie in our original dataframe
genreTable = filter_df.set_index(filter_df['anime_id'])
#And drop the unnecessary information
genreTable = genreTable.drop('anime_id', 1).drop('name', 1).drop('genre', 1).drop('type', 1).drop('episodes', 1).drop('members',1)
genreTable.head()

,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,Demons,Ecchi,Josei,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
32281,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [146]:
genreTable.shape

(12232, 82)

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.

In [147]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

anime_id
32281    0.568835
5114     0.000000
28977    0.000000
9253     0.000000
9969     0.000000
dtype: float64

In [148]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

anime_id
5653     0.586989
4010     0.586989
31118    0.586989
9904     0.586989
547      0.568835
dtype: float64

Now here's the recommendation table!

In [151]:
#The final recommendation table
filter_anime.loc[filter_anime['anime_id'].isin(recommendationTable_df.head(30).keys())]

,anime_id,name,genre,type,episodes,members
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",Movie,1,200630
10,4181,Clannad: After Story,"[Drama, Fantasy, Romance, Slice of Life, S...",TV,24,456749
180,18195,Little Busters!: Refrain,"[Comedy, Drama, Romance, School, Slice of ...",TV,13,71820
223,2167,Clannad,"[Comedy, Drama, Romance, School, Slice of ...",TV,23,566690
320,16001,Kokoro Connect: Michi Random,"[Comedy, Drama, Romance, School, Slice of ...",Special,4,106989
337,1530,Kanon (2006),"[Drama, Romance, Slice of Life, Supernatural]",TV,24,163171
518,11887,Kokoro Connect,"[Comedy, Drama, Romance, School, Slice of ...",TV,13,283847
613,20517,Little Busters!: EX,"[Comedy, Drama, Romance, School, Slice of ...",Special,8,26709
1616,101,Air,"[Drama, Romance, Slice of Life, Supernatural]",TV,13,188861
2058,1965,Sola,"[Drama, Romance, Slice of Life, Supernatural]",TV,13,71172


### Advantages and Disadvantages of Content-Based Filtering

##### Advantages
* Learns user's preferences
* Highly personalized for the user

##### Disadvantages
* Doesn't take into account what others think of the item, so low quality item recommendations might happen
* Extracting data is not always intuitive
* Determining what characteristics of the item the user dislikes or likes is not always obvious

## Want to learn more?

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at [Watson Studio](https://cocl.us/ML0101EN_DSX)

### Thanks for completing this lesson!

Notebook created by: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, Gabriel Garcez Barros Sousa

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​